In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [2]:
dataframe1=pd.read_csv('listings_summary.csv')
dataframe1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22552 entries, 0 to 22551
Data columns (total 96 columns):
id                                  22552 non-null int64
listing_url                         22552 non-null object
scrape_id                           22552 non-null int64
last_scraped                        22552 non-null object
name                                22493 non-null object
summary                             21589 non-null object
space                               14020 non-null object
description                         22349 non-null object
experiences_offered                 22552 non-null object
neighborhood_overview               11540 non-null object
notes                               7215 non-null object
transit                             13036 non-null object
access                              10837 non-null object
interaction                         10406 non-null object
house_rules                         11449 non-null object
thumbnail_url           

In [3]:
cls =   ['id', 'space', 'description', 'host_has_profile_pic', 'neighbourhood_group_cleansed', 
       'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',  
       'bedrooms', 'bed_type', 'amenities', 'square_feet', 'price', 'cleaning_fee', 
       'security_deposit', 'extra_people', 'guests_included', 'minimum_nights',  
       'instant_bookable', 'is_business_travel_ready', 'cancellation_policy']

dataframe2 = dataframe1[cls].set_index('id')

In [4]:
dataframe2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22552 entries, 2015 to 29867352
Data columns (total 23 columns):
space                           14020 non-null object
description                     22349 non-null object
host_has_profile_pic            22526 non-null object
neighbourhood_group_cleansed    22552 non-null object
latitude                        22552 non-null float64
longitude                       22552 non-null float64
property_type                   22552 non-null object
room_type                       22552 non-null object
accommodates                    22552 non-null int64
bathrooms                       22520 non-null float64
bedrooms                        22534 non-null float64
bed_type                        22552 non-null object
amenities                       22552 non-null object
square_feet                     446 non-null float64
price                           22552 non-null object
cleaning_fee                    15406 non-null object
security_deposit   

In [5]:
dataframe2.price.isnull().sum()

0

In [6]:
dataframe2.security_deposit.isnull().sum()
dataframe2.cleaning_fee.fillna('$0.00', inplace=True)
dataframe2.cleaning_fee.isnull().sum()
dataframe2.security_deposit.fillna('$0.00', inplace=True)
dataframe2.security_deposit.isnull().sum()

0

In [7]:
dataframe2.price = dataframe2.price.str.replace('$', '').str.replace(',', '').astype(float)
dataframe2.cleaning_fee = dataframe2.cleaning_fee.str.replace('$', '').str.replace(',', '').astype(float)
dataframe2.security_deposit = dataframe2.security_deposit.str.replace('$', '').str.replace(',', '').astype(float)
dataframe2.extra_people = dataframe2.extra_people.str.replace('$', '').str.replace(',', '').astype(float)

In [8]:
dataframe2['price'].describe()

count    22552.000000
mean        67.143668
std        220.266210
min          0.000000
25%         30.000000
50%         45.000000
75%         70.000000
max       9000.000000
Name: price, dtype: float64

In [9]:
dataframe2.drop(dataframe2[ (dataframe2.price > 400) | (dataframe2.price == 0) ].index, axis=0, inplace=True)

In [10]:
dataframe2['price'].describe()

count    22441.000000
mean        57.584377
std         42.162006
min          1.000000
25%         30.000000
50%         45.000000
75%         70.000000
max        400.000000
Name: price, dtype: float64

In [11]:
dataframe2.isnull().sum()

space                            8503
description                       202
host_has_profile_pic               25
neighbourhood_group_cleansed        0
latitude                            0
longitude                           0
property_type                       0
room_type                           0
accommodates                        0
bathrooms                          32
bedrooms                           18
bed_type                            0
amenities                           0
square_feet                     21996
price                               0
cleaning_fee                        0
security_deposit                    0
extra_people                        0
guests_included                     0
minimum_nights                      0
instant_bookable                    0
is_business_travel_ready            0
cancellation_policy                 0
dtype: int64

In [12]:
dataframe2.drop(['square_feet', 'space'], inplace=True,axis=1)

In [13]:
dataframe2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22441 entries, 2015 to 29867352
Data columns (total 21 columns):
description                     22239 non-null object
host_has_profile_pic            22416 non-null object
neighbourhood_group_cleansed    22441 non-null object
latitude                        22441 non-null float64
longitude                       22441 non-null float64
property_type                   22441 non-null object
room_type                       22441 non-null object
accommodates                    22441 non-null int64
bathrooms                       22409 non-null float64
bedrooms                        22423 non-null float64
bed_type                        22441 non-null object
amenities                       22441 non-null object
price                           22441 non-null float64
cleaning_fee                    22441 non-null float64
security_deposit                22441 non-null float64
extra_people                    22441 non-null float64
guests_include

In [14]:
dataframe2.dropna(subset=['bathrooms', 'bedrooms', ], inplace=True)

In [15]:
dataframe2.host_has_profile_pic.fillna(value='f', inplace=True)
dataframe2.host_has_profile_pic.unique()

array(['t', 'f'], dtype=object)

In [16]:
from geopy.distance import great_circle

In [17]:
def distance_to_mid(lat, lon):
    berlin_centre = (52.5027778, 13.404166666666667)
    accommodation = (lat, lon)
    return great_circle(berlin_centre, accommodation).km

In [18]:
dataframe2['distance'] = dataframe2.apply(lambda x: distance_to_mid(x.latitude, x.longitude), axis=1)

In [19]:
dataframe2['distance'].describe()

count    22391.000000
mean         4.547464
std          2.661159
min          0.198956
25%          2.979687
50%          4.088758
75%          5.551307
max         25.012109
Name: distance, dtype: float64

In [20]:
dataframe2['size'] = dataframe2['description'].str.extract('(\d{2,3}\s?[smSM])', expand=True)
dataframe2['size'] = dataframe2['size'].str.replace("\D", "")

# change datatype of size into float
dataframe2['size'] = dataframe2['size'].astype(float)

In [21]:
dataframe2.drop(['description'], axis=1, inplace=True)

In [22]:
dataframe3 = dataframe2[['accommodates', 'bathrooms', 'bedrooms',  'price', 'cleaning_fee', 
                 'security_deposit', 'extra_people', 'guests_included', 'distance', 'size']]

In [23]:
train_data = dataframe3[dataframe3['size'].notnull()]
test_data  = dataframe3[dataframe3['size'].isnull()]

# define X
X_train = train_data.drop('size', axis=1)
X_test  = test_data.drop('size', axis=1)

# define y
y_train = train_data['size']


In [24]:
from sklearn.linear_model import LinearRegression

# instantiate
linreg = LinearRegression()

# fit model to training data

linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [25]:
y_test = linreg.predict(X_test)

In [26]:
y_test = pd.DataFrame(y_test)
y_test.columns = ['size']
print(y_test.shape)

(11721, 1)


In [27]:
prelim_index = pd.DataFrame(X_test.index)
prelim_index.columns = ['prelim']

# ... and concat this dataframe with y_test
y_test = pd.concat([y_test, prelim_index], axis=1)
y_test.set_index(['prelim'], inplace=True)

In [28]:
new_test_data = pd.concat([X_test, y_test], axis=1)

In [29]:
new_test_data['size'].isnull().sum()

0

In [30]:
sub_df_new = pd.concat([new_test_data, train_data], axis=0)

print(sub_df_new.shape)
sub_df_new.head()

(22391, 10)


,accommodates,bathrooms,bedrooms,price,cleaning_fee,security_deposit,extra_people,guests_included,distance,size
14325,2,1.0,0.0,70.0,0.0,500.0,0.0,1,5.012297,56.697281
20858,4,1.0,2.0,129.0,49.0,500.0,24.0,2,3.807207,97.756906
21869,4,1.0,1.0,70.0,40.0,500.0,18.0,2,2.061292,75.831592
22415,4,1.0,2.0,98.0,50.0,300.0,25.0,2,1.868255,88.778838
28156,2,1.0,0.0,70.0,0.0,1660.0,0.0,1,4.886509,60.680846


In [31]:
dataframe2.drop(['accommodates', 'bathrooms', 'bedrooms', 'price', 'cleaning_fee', 
             'security_deposit', 'extra_people', 'guests_included', 'distance', 'size'], 
            axis=1, inplace=True)

In [32]:
df = pd.concat([sub_df_new, dataframe2], axis=1)

print(df.shape)
df.head(2)

(22391, 22)


,accommodates,bathrooms,bedrooms,price,cleaning_fee,security_deposit,extra_people,guests_included,distance,size,...,latitude,longitude,property_type,room_type,bed_type,amenities,minimum_nights,instant_bookable,is_business_travel_ready,cancellation_policy
2015,3,1.0,1.0,60.0,30.0,200.0,28.0,1,3.533182,75.0,...,52.534537,13.402557,Guesthouse,Entire home/apt,Real Bed,"{TV,""Cable TV"",Wifi,Kitchen,Gym,Heating,""Famil...",4,f,f,strict_14_with_grace_period
2695,2,1.0,1.0,17.0,0.0,0.0,0.0,1,5.085573,25.0,...,52.548513,13.404553,Apartment,Private room,Real Bed,"{Wifi,Kitchen,Elevator,Heating,Washer,Essentia...",2,f,f,flexible


In [33]:
df['size'].isnull().sum()

0

In [34]:
df.drop(df[ (df['size'] == 0.) | (df['size'] > 300.) ].index, axis=0, inplace=True)

In [35]:
from collections import Counter

In [36]:
results = Counter()
df['amenities'].str.strip('{}')\
               .str.replace('"', '')\
               .str.lstrip('\"')\
               .str.rstrip('\"')\
               .str.split(',')\
               .apply(results.update)

results.most_common(30)


[('Wifi', 20952),
 ('Kitchen', 20809),
 ('Heating', 20657),
 ('Essentials', 19811),
 ('Washer', 18071),
 ('Hair dryer', 14474),
 ('Laptop friendly workspace', 14095),
 ('Hangers', 13514),
 ('Iron', 10876),
 ('Shampoo', 10339),
 ('TV', 9778),
 ('Hot water', 9421),
 ('Family/kid friendly', 7925),
 ('Internet', 7699),
 ('Host greets you', 6469),
 ('Smoke detector', 6245),
 ('Buzzer/wireless intercom', 5963),
 ('Lock on bedroom door', 5138),
 ('Free street parking', 5037),
 ('Refrigerator', 5033),
 ('Dishes and silverware', 4909),
 ('Elevator', 4703),
 ('Cooking basics', 4673),
 ('Bed linens', 4656),
 ('Stove', 4655),
 ('Smoking allowed', 4509),
 ('Oven', 4161),
 ('translation missing: en.hosting_amenity_50', 4095),
 ('First aid kit', 3866),
 ('Cable TV', 3577)]

In [37]:
sub_df = pd.DataFrame(results.most_common(30), columns=['amenity', 'count'])

In [38]:
df['Laptop_friendly_workspace'] = df['amenities'].str.contains('Laptop friendly workspace')
df['TV'] = df['amenities'].str.contains('TV')
df['Family_kid_friendly'] = df['amenities'].str.contains('Family/kid friendly')
df['Host_greets_you'] = df['amenities'].str.contains('Host greets you')
df['Smoking_allowed'] = df['amenities'].str.contains('Smoking allowed')

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21958 entries, 2015 to 29867352
Data columns (total 27 columns):
accommodates                    21958 non-null int64
bathrooms                       21958 non-null float64
bedrooms                        21958 non-null float64
price                           21958 non-null float64
cleaning_fee                    21958 non-null float64
security_deposit                21958 non-null float64
extra_people                    21958 non-null float64
guests_included                 21958 non-null int64
distance                        21958 non-null float64
size                            21958 non-null float64
host_has_profile_pic            21958 non-null object
neighbourhood_group_cleansed    21958 non-null object
latitude                        21958 non-null float64
longitude                       21958 non-null float64
property_type                   21958 non-null object
room_type                       21958 non-null object
bed_type     

In [40]:
df.drop(['amenities'], axis=1, inplace=True)

In [41]:
df_grouped = pd.DataFrame(df.groupby(['neighbourhood_group_cleansed'])['price'].agg(np.median))
df_grouped.reset_index(inplace=True)

In [42]:
df_grouped = pd.DataFrame(df.groupby(['cancellation_policy'])['price'].agg(np.median))
df_grouped.reset_index(inplace=True)

In [43]:
df.drop(['latitude', 'longitude', 'neighbourhood_group_cleansed', 'property_type'], axis=1, inplace=True)

In [44]:
for col in ['host_has_profile_pic', 'room_type', 'bed_type', 'instant_bookable', 
            'is_business_travel_ready', 'cancellation_policy']:
    df[col] = df[col].astype('category')

In [45]:
target = df[["price"]]

# define our features 
features = df.drop(["price"], axis=1)

In [46]:
num_feats = features.select_dtypes(include=['float64', 'int64', 'bool']).copy()

# one-hot encoding of categorical features
cat_feats = features.select_dtypes(include=['category']).copy()
cat_feats = pd.get_dummies(cat_feats)

In [47]:
features_recoded = pd.concat([num_feats, cat_feats], axis=1)
print(features_recoded.shape)
features_recoded.head(2)

(21958, 33)


,accommodates,bathrooms,bedrooms,cleaning_fee,security_deposit,extra_people,guests_included,distance,size,minimum_nights,...,bed_type_Pull-out Sofa,bed_type_Real Bed,instant_bookable_f,instant_bookable_t,is_business_travel_ready_f,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60
2015,3,1.0,1.0,30.0,200.0,28.0,1,3.533182,75.0,4,...,0,1,1,0,1,0,0,1,0,0
2695,2,1.0,1.0,0.0,0.0,0.0,1,5.085573,25.0,2,...,0,1,1,0,1,1,0,0,0,0


In [48]:
# import train_test_split function
from sklearn.model_selection import train_test_split
# import metrics
from sklearn.metrics import mean_squared_error, r2_score

# split our data
X_train, X_test, y_train, y_test = train_test_split(features_recoded, target, test_size=0.2)

In [49]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test  = sc.transform(X_test)

In [50]:
# create a baseline
booster = xgb.XGBRegressor()

In [115]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [150, 200, 250],
              'learning_rate': [0.05, 0.1, 0.15], 
              'max_depth': [ 6, 7, 8, 9],
              'colsample_bytree': [ 0.6, 0.7, 0.8],
              'gamma': [ 0.1, 0.2,0.22]}

# instantiate the tuned random forest
booster_grid_search = GridSearchCV(booster, param_grid, cv=3, n_jobs=-1)

# train the tuned random forest
booster_grid_search.fit(X_train, y_train)

# print best estimator parameters found during the grid search
print(booster_grid_search.best_params_)

{'colsample_bytree': 0.6, 'gamma': 0.22, 'learning_rate': 0.05, 'max_depth': 9, 'n_estimators': 250}


In [93]:
booster = xgb.XGBRegressor(colsample_bytree=0.6, gamma=0.24, learning_rate=0.05, 
                           max_depth=8, n_estimators=550, random_state=4)

# train
booster.fit(X_train, y_train)

# predict
y_pred_train = booster.predict(X_train)
y_pred_test = booster.predict(X_test)

In [94]:
RMSE = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f"RMSE: {round(RMSE, 4)}")

RMSE: 22.0433


In [95]:
r2 = r2_score(y_test, y_pred_test)
r2
print(f"r2: {round(r2, 4)}")

r2: 0.7199
